====== Nguồn http://users.soict.hust.edu.vn/khoattq/ml-dm-course/ ======



## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files


%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [3]:
INPUT = 'data/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh kết quả trong quá trình huấn luyện và đánh giá

In [4]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 122
du-lich: 57
giai-tri: 196
giao-duc: 118
khoa-hoc: 133
kinh-doanh: 229
phap-luat: 26
suc-khoe: 193
the-thao: 177
thoi-su: 58
-------------------------
Tổng số văn bản: 1309


In [6]:
# load data
#Load dữ liệu văn bản từ thư mục được chỉ định bởi biến INPUT 
# sử dụng hàm load_files từ module datasets trong scikit-learn.
#In ra thống kê về dữ liệu, bao gồm số lượng văn bản cho mỗi nhãn.
#Ánh xạ các nhãn với các chỉ số tương ứng.
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
print(data_train.data[0:1])
#print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['data/news_vnexpress\\the-thao\\00065.txt']
['Cú chip đầu diễn ra ở hố 15 par4, khi Matsuyama đưa bóng vào bãi cỏ rough dày, cách lỗ khoảng 8,5 mét. Do anh nhẹ tay, bóng bay trước nhưng lại bị đầu gậy sắt bắt kịp. Vì thế, cựu vô địch major người Nhật Bản tình cờ có "một cú hai chạm". Dù vậy, bóng chỉ đến cách mục tiêu hơn ba mét. Từ đó, Matsuyama ghi bogey sau hai gạt.Cú chip thứ hai diễn ra tại hố 16 par5, dẫn đến eagle khi Matsuyama ra tay cách lỗ 15,2 mét. Nhờ vậy, anh lên điểm vòng -5. Khi Matsuyama ghi par cả hai hố còn lại, nó trở thành điểm giải cùng vị trí T2 cho anh. Đỉnh bảng thuộc về Shane Lowry ở mức -6.Kết quả vòng 1 của Matsuyama hẳn đã khác, nếu theo chiếu luật golf cũ khi anh chạm bóng hai lần trong cú chip tại hố 15. Trong tình huống như thế, luật từ 2018 trở về trước sẽ phạt hai gậy, còn hiện nay thì kh

## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

In [7]:
# load dữ liệu các stopwords 
# Mở tệp chứa danh sách các stopwords tiếng Việt bằng cách sử dụng open() với encoding "utf8".
#Đọc danh sách các từ stopwords từ tệp và lưu chúng vào danh sách stopwords.
with open("data/vietnamese-stopwords.txt", encoding="utf8") as f:
    stopwords = f.readlines()
#Loại bỏ các khoảng trắng thừa và thay thế bằng dấu gạch dưới "_" để chuẩn hóa.
stopwords = [x.strip().replace(" ", "_") for x in stopwords] 
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])

# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng
#     - sinh từ điển
# Khởi tạo CountVectorizer:

# CountVectorizer được khởi tạo với tham số stop_words=stopwords, nơi stopwords là danh sách các từ dừng được loại bỏ trước đó.
# CountVectorizer này sẽ chuyển đổi dữ liệu văn bản thành ma trận token counts, bỏ qua các từ trong danh sách stopwords.
# Khởi tạo TfidfTransformer:

# TfidfTransformer được khởi tạo để thực hiện việc chuyển đổi ma trận token counts thành ma trận TF-IDF.
# Kết hợp CountVectorizer và TfidfTransformer vào một Pipeline:

# Sử dụng Pipeline từ scikit-learn để gom CountVectorizer và TfidfTransformer thành một chuỗi các bước tiền xử lý dữ liệu.
# Trong Pipeline, CountVectorizer được gán với tên 'vect' và TfidfTransformer được gán với tên 'tfidf'.
# Kết quả của bước tiền xử lý này sẽ là ma trận TF-IDF của dữ liệu văn bản đã được xử lý.

module_count_vector = CountVectorizer(stop_words=stopwords)
model_rf_preprocess = Pipeline([('vect', module_count_vector),
                    ('tfidf', TfidfTransformer()),
                    ])
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data, data_train.target)

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {data_preprocessed.shape}")
print(f"Kích thước nhãn tương ứng: {data_train.target.shape}")

Số lượng stopwords: 2063
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 14444
Kích thước dữ liệu sau khi xử lý: (1309, 14444)
Kích thước nhãn tương ứng: (1309,)


In [10]:
X = data_preprocessed
Y = data_train.target

In [11]:
X.shape, Y.shape

((1309, 14444), (1309,))

In [10]:
# in ra biểu diễn ma trận của văn bản thứ 100 trong dữ liệu, 
# được chuyển đổi về dạng mảng 1 chiều bằng phương thức toarray(). Biểu diễn này là ma trận TF-IDF của văn bản đó.
print(X[100].toarray())
#in ra nhãn tương ứng của văn bản thứ 100 trong dữ liệu. Điều này sẽ cho bạn biết văn bản đó thuộc vào nhóm lớp nào.
print(Y[100])

[[0.         0.         0.         ... 0.04842098 0.         0.        ]]
4


In [30]:
#Tính tổng số lượng từ có giá trị khác 0
#tiếp tục tính tổng của các tổng này, tức là tổng số lượng từ khác 0 trong tất cả các văn bản.
sum(sum(X[100].toarray() != 0))

182

In [31]:
print(X[100])

  (0, 14441)	0.04842097684377052
  (0, 14427)	0.23507525989082878
  (0, 14423)	0.0385754970377109
  (0, 14403)	0.07334040426970828
  (0, 14364)	0.10882963057484636
  (0, 14360)	0.013038098578746972
  (0, 14356)	0.015479383510224534
  (0, 14347)	0.03185993747182163
  (0, 14341)	0.020694193640872594
  (0, 14293)	0.019308551389366133
  (0, 14289)	0.03811305614790038
  (0, 14276)	0.019101674687790175
  (0, 14267)	0.013151083732313558
  (0, 14260)	0.015204197748574703
  (0, 14205)	0.021585024208002953
  (0, 14203)	0.046228757767075156
  (0, 14195)	0.020478999052021486
  (0, 14159)	0.07526466702814646
  (0, 14127)	0.09693825686327658
  (0, 14118)	0.031038716838191358
  (0, 14094)	0.03521777988626455
  (0, 14011)	0.01663820851103926
  (0, 13880)	0.32386979281639544
  (0, 13869)	0.06968241538513387
  (0, 13769)	0.38131160205374415
  :	:
  (0, 3447)	0.009807867666120016
  (0, 3021)	0.032435378955159834
  (0, 3019)	0.04598855709095791
  (0, 3016)	0.026913121774857936
  (0, 3003)	0.07804423912151